## Loading Necessary Libraries

In [1]:
# general
import re
import gc
import csv
import time
import pickle
from tqdm import tqdm

# data handling
import numpy as np
import pandas as pd

# HTML parsing
from bs4 import BeautifulSoup

# plotting
import matplotlib.pyplot as plt

# chunker
import torch
from torch import mps
import flair
from flair.data import Sentence
from flair.models import SequenceTagger

## Load Data

In [2]:
# 2023 Data
# data = pd.read_csv('../data/2023/train-v2.csv')
# data = pd.read_csv('../data/2023/test-v2.csv')

# 2021 Data
# data = pd.read_csv('../data/2021/train-v2.csv')
data = pd.read_csv('../data/2021/test-v2.csv')

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_5114/1659300443.py:7: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/2021/test-v2.csv')


In [3]:
data = data.fillna('')

In [4]:
data

,PRODUCT_ID,TITLE,DESCRIPTION,BRAND,0,1,2,3,4,5,...,38,39,40,41,42,43,44,45,46,47
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,Command,INCLUDES - 9 hooks and 12 small indoor strips;...,"ORGANIZE DAMAGE-FREE - Say goodbye to holes, m...","NO TOOLS REQUIRED- Hang hats, calendars, keys,...",STRONG AND VERSATILE- Command Wire Hooks inclu...,REMOVES CLEANLY- Reorganize when inspiration s...,Damage-Free Hanging,...,,,,,,,,,,
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,O'Neal,Silicone printing for a better grip. Long last...,Super lightweight design with bold non-fade gr...,Seamless rollover fingertips. Synthetic leathe...,,,,...,,,,,,,,,,
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,Boelter Brands,Runs on 12 Volt DC Power or 110 Volt AC Power;...,Holds up to 24 regular 12-ounce cans,Vertical Space for 1.5 Liter Bottles,Decorated with Vibrant Team Colors,Thermo-Electric Fridge Not for Continuous Use;...,,...,,,,,,,,,,
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic,Panasonic Landline Phones doesn't come with a ...,,,,,,...,,,,,,,,,,
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...",Zero,"Zero Baby Girl Panties, Pack of 6, 100% Cotton...","Set Includes (6) Panties (Drawers, Panties), C...","Ideal For Baby Girls, Innerwear for Winter, Su...",Wash Care: Easy to Wash and Dry. Can be Hand W...,Very Soft and Feature Adorable and Fun Designs...,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110770,110771,AAHNA E MALL OneBlade Hybrid Trimmer Shaver An...,1-All In One Hyper Advanced Smart Rechargeable...,Generic,"Unique One Blade can style, trim and shave, wh...",Completely safe to the touch,Cordless: 180 minute charge time for 45 minute...,German stainless steel dual-sided blade cuts t...,"All-in-one trimmer, shaver and edger with a bu...","Includes three trimming combs (1mm, 3mm, 5mm),...",...,,,,,,,,,,
110771,110772,Grin Health N99 Anti Pollution Reusable Washab...,"SIZE GUIDE : M - (35- 65 Kg), L - (49- 72 Kg),...",Grin Health,PROTECTION: Filtration rate up to ≥99 percent ...,ACTIVATED CARBON: Activated carbon layer provi...,"SEALED, STYLISH, SMART: New generation design....",GHAirVALVE: Valve allow CO2 and moisture to es...,"SIZE GUIDE : M - (35- 65 Kg), L - (49- 72 Kg),...",,...,,,,,,,,,,
110772,110773,Asian Army Pink Ultra reusable respirator clot...,Asian HyperProtect A95 masks have been enginee...,ASIAN,Reusable and environment friendly: These masks...,Technology: Made in India | Engineered with a ...,Use Cases: This pocket friendly reusable face ...,About Asian: We are an ISO 9001: 2015 certifie...,"000+ reviews, offering the latest best quality...",Friendly Customer Service: What you see is wha...,...,,,,,,,,,,
110773,110774,IM Safe 3 Ply Non-Woven Disposable Surgical Fa...,This 3 Ply Disposable face mask is manufacture...,Intermarket,3 Ply Mask: Genuine 3 Ply Mask. 25 GSM Spun Bo...,TOP Quality: These face masks are soft & comfo...,MADE IN INDIA: Directly from the Manufacturer....,PACK OF 50 PCS: The inner packet contains 10 p...,"CERTIFICATION: We, at InterMarket (India) Pvt....",,...,,,,,,,,,,


## Regex Cleaning

In [62]:
def replace_non_alphanumeric(text):
  pattern = r"[\w^-]+|[^a-zA-Z0-9 ]"
  replaced_text = ' '.join(re.findall(pattern, text))
  replaced_text = ' '.join(re.split(r'_', replaced_text))
  return replaced_text

def merge_decimal(text):
  pattern = r'(\d) \. (\d)'
  replaced_text = re.sub(pattern, r'\1.\2', text)
  return replaced_text

def replace_double_quote_with_inch(text):
  pattern = r'(\d) "|(\d) ”'
  replaced_text = re.sub(pattern, r'\1 inch', text)
  return replaced_text
  
def replace_two_single_quote_with_inch(text):
  pattern = r'(\d) (\' \')'
  replaced_text = re.sub(pattern, r'\1 inch', text)
  return replaced_text

def replace_hyphen_with_to(text):
  pattern = r'(?<=\d)\s*-\s*(?=\d)'
  replaced_text = re.sub(pattern, ' to ', text)
  return replaced_text

def replace_slash_with_or(text):
  pattern = r'[\\/]'
  replaced_text = re.sub(pattern, ' or ', text)
  return replaced_text

def replace_vertical_bar_with_comma(text):
  pattern = r'\|'
  replaced_text = re.sub(pattern, ' , ', text)
  return replaced_text

def replace_amp_with_and(text):
  pattern = r'\&'
  replaced_text = re.sub(pattern, ' and ', text)
  return replaced_text

def replace_hypen_with_space(text):
  return ''.join(text.split(' - '))

def replace_multiple_spaces(text):
  pattern = r'\s+'
  replaced_text = re.sub(pattern, ' ', text)
  return replaced_text

def split_alphabet_number(text):
  # pattern = r'(?<=[a-zA-Z])(?=\d)|(?<=\d)(?=[a-zA-Z])'
  # replaced_text = ' '.join(re.split(pattern, text))
  replaced_text = text
  return replaced_text

def replace_pieces(text):
    pattern = r'\b(PC|Pcs|pcs|PCS|pc|Pc)\b'
    replaced_text = re.sub(pattern, lambda match: 'pieces' if match.group().lower() == 'pcs' else 'piece', text)
    return replaced_text
  
def replace_years(text):
    pattern = r'\b(Yrs|yrs|YRS|yr|Yr)\b'
    replaced_text = re.sub(pattern, lambda match: 'years' if match.group().lower() == 'yrs' else 'yr', text)
    return replaced_text
  
def regex_cleaning(text):
  replaced_text = replace_multiple_spaces(
    replace_years(
      replace_pieces(
        merge_decimal(
          replace_double_quote_with_inch(
            replace_two_single_quote_with_inch(
              split_alphabet_number(
                replace_hypen_with_space(
                  replace_slash_with_or(
                    replace_vertical_bar_with_comma(
                      replace_amp_with_and(
                        replace_hyphen_with_to(
                          replace_non_alphanumeric(text)
                        )
                      )
                    )
                  )
                )
              )
            )
          )
        )
      )
    )
  )
  return replaced_text

In [63]:
data.TITLE = data.TITLE.apply(lambda x: regex_cleaning(x))
data.DESCRIPTION = data.DESCRIPTION.apply(lambda x: regex_cleaning(x))

In [64]:
data

,PRODUCT_ID,TITLE,DESCRIPTION,BRAND,0,1,2,3,4,5,...,38,39,40,41,42,43,44,45,46,47
0,1,"Command 3M Small Kitchen Hooks , White , Decor...",Sale Unit : PACK,Command,INCLUDES - 9 hooks and 12 small indoor strips;...,"ORGANIZE DAMAGE-FREE - Say goodbye to holes, m...","NO TOOLS REQUIRED- Hang hats, calendars, keys,...",STRONG AND VERSATILE- Command Wire Hooks inclu...,REMOVES CLEANLY- Reorganize when inspiration s...,Damage-Free Hanging,...,,,,,,,,,,
1,2,O ' Neal Jump Hardware JAG Unisex-Adult Glove ...,Synthetic leather palm with double-layer thumb...,O'Neal,Silicone printing for a better grip. Long last...,Super lightweight design with bold non-fade gr...,Seamless rollover fingertips. Synthetic leathe...,,,,...,,,,,,,,,,
2,3,"NFL Detroit Lions Portable Party Fridge , 15.8...",Boelter Brands lets you celebrate your favorit...,Boelter Brands,Runs on 12 Volt DC Power or 110 Volt AC Power;...,Holds up to 24 regular 12-ounce cans,Vertical Space for 1.5 Liter Bottles,Decorated with Vibrant Team Colors,Thermo-Electric Fridge Not for Continuous Use;...,,...,,,,,,,,,,
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features : 50 Station Phonebook Corded Phone A...,Panasonic,Panasonic Landline Phones doesn't come with a ...,,,,,,...,,,,,,,,,,
4,5,Zero Baby Girl ' s 100 % Cotton Innerwear Bloo...,"Zero Baby Girl Panties Set . 100 % Cotton , Br...",Zero,"Zero Baby Girl Panties, Pack of 6, 100% Cotton...","Set Includes (6) Panties (Drawers, Panties), C...","Ideal For Baby Girls, Innerwear for Winter, Su...",Wash Care: Easy to Wash and Dry. Can be Hand W...,Very Soft and Feature Adorable and Fun Designs...,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110770,110771,AAHNA E MALL OneBlade Hybrid Trimmer Shaver An...,1-All In One Hyper Advanced Smart Rechargeable...,Generic,"Unique One Blade can style, trim and shave, wh...",Completely safe to the touch,Cordless: 180 minute charge time for 45 minute...,German stainless steel dual-sided blade cuts t...,"All-in-one trimmer, shaver and edger with a bu...","Includes three trimming combs (1mm, 3mm, 5mm),...",...,,,,,,,,,,
110771,110772,Grin Health N99 Anti Pollution Reusable Washab...,"SIZE GUIDE : M( 35 to 65 Kg ) , L( 49 to 72 Kg...",Grin Health,PROTECTION: Filtration rate up to ≥99 percent ...,ACTIVATED CARBON: Activated carbon layer provi...,"SEALED, STYLISH, SMART: New generation design....",GHAirVALVE: Valve allow CO2 and moisture to es...,"SIZE GUIDE : M - (35- 65 Kg), L - (49- 72 Kg),...",,...,,,,,,,,,,
110772,110773,Asian Army Pink Ultra reusable respirator clot...,Asian HyperProtect A95 masks have been enginee...,ASIAN,Reusable and environment friendly: These masks...,Technology: Made in India | Engineered with a ...,Use Cases: This pocket friendly reusable face ...,About Asian: We are an ISO 9001: 2015 certifie...,"000+ reviews, offering the latest best quality...",Friendly Customer Service: What you see is wha...,...,,,,,,,,,,
110773,110774,IM Safe 3 Ply Non-Woven Disposable Surgical Fa...,This 3 Ply Disposable face mask is manufacture...,Intermarket,3 Ply Mask: Genuine 3 Ply Mask. 25 GSM Spun Bo...,TOP Quality: These face masks are soft & comfo...,MADE IN INDIA: Directly from the Manufacturer....,PACK OF 50 PCS: The inner packet contains 10 p...,"CERTIFICATION: We, at InterMarket (India) Pvt....",,...,,,,,,,,,,


In [65]:
data_bps = data.iloc[:,4:].to_numpy().tolist()

In [66]:
data_bps = [
  [bp for bp in bps if bp != ''] 
  for bps in data_bps 
]

In [67]:
data_bps_len = [len(bps) for bps in data_bps]

In [68]:
temp_bps = pd.Series([bp for bps in data_bps for bp in bps]).astype(str)

In [69]:
temp_bps = temp_bps.apply(lambda x: regex_cleaning(x))

In [70]:
data_bps_cut_offs = np.cumsum([0]+data_bps_len)

In [71]:
temp_bps = temp_bps.to_list()

In [72]:
replace_bps = [temp_bps[data_bps_cut_offs[i]:data_bps_cut_offs[i+1]] for i in range(len(data_bps_cut_offs)-1)]

In [73]:
replace_bps = pd.DataFrame(replace_bps).fillna('')

In [74]:
replace_bps

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,INCLUDES9 hooks and 12 small indoor strips ; 1...,"ORGANIZE DAMAGE-FREESay goodbye to holes , mar...","NO TOOLS REQUIRED- Hang hats , calendars , key...",STRONG AND VERSATILE- Command Wire Hooks inclu...,REMOVES CLEANLY- Reorganize when inspiration s...,Damage-Free Hanging,Holds strongly,Removes cleanly,Easy to apply and remove,Works on a variety of surfaces,...,,,,,,,,,,
1,Silicone printing for a better grip . Long las...,Super lightweight design with bold non-fade gr...,Seamless rollover fingertips . Synthetic leath...,,,,,,,,...,,,,,,,,,,
2,Runs on 12 Volt DC Power or 110 Volt AC Power ...,Holds up to 24 regular 12-ounce cans,Vertical Space for 1.5 Liter Bottles,Decorated with Vibrant Team Colors,Thermo-Electric Fridge Not for Continuous Use ...,,,,,,...,,,,,,,,,,
3,Panasonic Landline Phones doesn ' t come with ...,,,,,,,,,,...,,,,,,,,,,
4,"Zero Baby Girl Panties , Pack of 6 , 100 % Cot...","Set Includes ( 6 ) Panties ( Drawers , Panties...","Ideal For Baby Girls , Innerwear for Winter , ...",Wash Care : Easy to Wash and Dry . Can be Hand...,Very Soft and Feature Adorable and Fun Designs...,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110770,"Unique One Blade can style , trim and shave , ...",Completely safe to the touch,Cordless : 180 minute charge time for 45 minut...,German stainless steel dual-sided blade cuts t...,"All-in-one trimmer , shaver and edger with a b...","Includes three trimming combs ( 1mm , 3mm , 5m...",,,,,...,,,,,,,,,,
110771,PROTECTION : Filtration rate up to ≥ 99 percen...,ACTIVATED CARBON : Activated carbon layer prov...,"SEALED , STYLISH , SMART : New generation desi...",GHAirVALVE : Valve allow CO2 and moisture to e...,"SIZE GUIDE : M( 35 to 65 Kg ) , L( 49 to 72 Kg...",,,,,,...,,,,,,,,,,
110772,Reusable and environment friendly : These mask...,"Technology : Made in India , Engineered with a...",Use Cases : This pocket friendly reusable face...,About Asian : We are an ISO 9001 : 2015 certif...,"000 + reviews , offering the latest best quali...",Friendly Customer Service : What you see is wh...,,,,,...,,,,,,,,,,
110773,3 Ply Mask : Genuine 3 Ply Mask . 25 GSM Spun ...,TOP Quality : These face masks are soft and co...,MADE IN INDIA : Directly from the Manufacturer...,PACK OF 50 pieces : The inner packet contains ...,"CERTIFICATION : We , at InterMarket ( India ) ...",,,,,,...,,,,,,,,,,


In [75]:
data = pd.concat([data.iloc[:,:4], replace_bps], axis=1)

In [76]:
data

,PRODUCT_ID,TITLE,DESCRIPTION,BRAND,0,1,2,3,4,5,...,38,39,40,41,42,43,44,45,46,47
0,1,"Command 3M Small Kitchen Hooks , White , Decor...",Sale Unit : PACK,Command,INCLUDES9 hooks and 12 small indoor strips ; 1...,"ORGANIZE DAMAGE-FREESay goodbye to holes , mar...","NO TOOLS REQUIRED- Hang hats , calendars , key...",STRONG AND VERSATILE- Command Wire Hooks inclu...,REMOVES CLEANLY- Reorganize when inspiration s...,Damage-Free Hanging,...,,,,,,,,,,
1,2,O ' Neal Jump Hardware JAG Unisex-Adult Glove ...,Synthetic leather palm with double-layer thumb...,O'Neal,Silicone printing for a better grip . Long las...,Super lightweight design with bold non-fade gr...,Seamless rollover fingertips . Synthetic leath...,,,,...,,,,,,,,,,
2,3,"NFL Detroit Lions Portable Party Fridge , 15.8...",Boelter Brands lets you celebrate your favorit...,Boelter Brands,Runs on 12 Volt DC Power or 110 Volt AC Power ...,Holds up to 24 regular 12-ounce cans,Vertical Space for 1.5 Liter Bottles,Decorated with Vibrant Team Colors,Thermo-Electric Fridge Not for Continuous Use ...,,...,,,,,,,,,,
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features : 50 Station Phonebook Corded Phone A...,Panasonic,Panasonic Landline Phones doesn ' t come with ...,,,,,,...,,,,,,,,,,
4,5,Zero Baby Girl ' s 100 % Cotton Innerwear Bloo...,"Zero Baby Girl Panties Set . 100 % Cotton , Br...",Zero,"Zero Baby Girl Panties , Pack of 6 , 100 % Cot...","Set Includes ( 6 ) Panties ( Drawers , Panties...","Ideal For Baby Girls , Innerwear for Winter , ...",Wash Care : Easy to Wash and Dry . Can be Hand...,Very Soft and Feature Adorable and Fun Designs...,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110770,110771,AAHNA E MALL OneBlade Hybrid Trimmer Shaver An...,1-All In One Hyper Advanced Smart Rechargeable...,Generic,"Unique One Blade can style , trim and shave , ...",Completely safe to the touch,Cordless : 180 minute charge time for 45 minut...,German stainless steel dual-sided blade cuts t...,"All-in-one trimmer , shaver and edger with a b...","Includes three trimming combs ( 1mm , 3mm , 5m...",...,,,,,,,,,,
110771,110772,Grin Health N99 Anti Pollution Reusable Washab...,"SIZE GUIDE : M( 35 to 65 Kg ) , L( 49 to 72 Kg...",Grin Health,PROTECTION : Filtration rate up to ≥ 99 percen...,ACTIVATED CARBON : Activated carbon layer prov...,"SEALED , STYLISH , SMART : New generation desi...",GHAirVALVE : Valve allow CO2 and moisture to e...,"SIZE GUIDE : M( 35 to 65 Kg ) , L( 49 to 72 Kg...",,...,,,,,,,,,,
110772,110773,Asian Army Pink Ultra reusable respirator clot...,Asian HyperProtect A95 masks have been enginee...,ASIAN,Reusable and environment friendly : These mask...,"Technology : Made in India , Engineered with a...",Use Cases : This pocket friendly reusable face...,About Asian : We are an ISO 9001 : 2015 certif...,"000 + reviews , offering the latest best quali...",Friendly Customer Service : What you see is wh...,...,,,,,,,,,,
110773,110774,IM Safe 3 Ply Non-Woven Disposable Surgical Fa...,This 3 Ply Disposable face mask is manufacture...,Intermarket,3 Ply Mask : Genuine 3 Ply Mask . 25 GSM Spun ...,TOP Quality : These face masks are soft and co...,MADE IN INDIA : Directly from the Manufacturer...,PACK OF 50 pieces : The inner packet contains ...,"CERTIFICATION : We , at InterMarket ( India ) ...",,...,,,,,,,,,,


In [77]:
# data.to_csv('../Data/2023/train-v3.csv')
# data.to_csv('../Data/2023/test-v3.csv')

# data.to_csv('../Data/2021/train-v3.csv')
data.to_csv('../Data/2021/test-v3.csv')

In [ ]:
# data_test_old

In [53]:
# data_test_old[['TITLE']] = data_test_old[['TITLE']].astype(str)

## Additional cleaning

In [14]:
# 2023 Data
data = pd.read_csv('../data/2023/train-v3.csv').iloc[:,1:]
# data = pd.read_csv('../data/2023/test-v3.csv').iloc[:,1:]

# 2021 Data
# data = pd.read_csv('../data/2021/train-v3.csv').iloc[:,1:]
# data = pd.read_csv('../data/2021/test-v3.csv').iloc[:,1:]

data = data.fillna('')

/var/folders/mg/l90d6s9536n6mfzjbqgrxffm0000gn/T/ipykernel_5149/4011170747.py:2: DtypeWarning: Columns (27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/2023/train-v3.csv').iloc[:,1:]


In [15]:
data

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,0,1,2,3,4,...,183,184,185,186,187,188,189,190,191,192
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,,1650,2125.980000,LUXURIOUS and APPEALING : Beautiful custom-mad...,MATERIAL : Luxurious and versatile fabric with...,BLACKOUT CURTAIN : 100 % opaque and heavy prem...,MADE TO PERFECTION : Large eyelets at the top ...,QUALITY ASSURED : Gentle wash with similar col...,...,,,,,,,,,,
1,2673191,Marks and Spencer Girls ' Pyjama Sets T86 2561...,,2755,393.700000,Harry Potter Hedwig Pyjamas ( 6 to 16 years ),100 % cotton ( exclusive of trimmings ),"Ribbing98 % cotton , 2 % elastane",,,...,,,,,,,,,,
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Specifications : Color : Red , Material : Alum...",7537,748.031495,"Loud Dual Tone Trumpet Horn , Compatible With SX4","Electric Air Horn with Compressor , Compatible...","High Quality , High Intensity Horn Sound , Com...","Color : Red , Compatible With SX4","Professional installation required , Compatibl...",...,,,,,,,,,,
3,1594019,ALISHAH Women ' s Cotton Ankle Length Leggings...,AISHAH Women ' s Lycra Cotton Ankel Leggings ....,2996,787.401574,Made By 95 % cotton and 5 % Lycra which gives ...,"Come with Elasticized Waist Band , Slim Fit",Bubble free fabric and High quality steaching ...,Skin-friendly fabric,ANKLE LENGTH LEGGINGS PACK OF 2,...,,,,,,,,,,
4,283658,The United Empire Loyalists : A Chronicle of t...,,6112,598.424000,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249693,2422167,Nike Women ' s As W Ny Df Swsh Hn Kh Bra ( CZ7...,,3009,1181.100000,Material : Polyester,,,,,...,,,,,,,,,,
2249694,2766635,"( 3PCS ) Goose Game Cute Cartoon Enamel Pins ,...",[ Brand ] : XVIEONR [ Product name ] : Fashion...,3413,125.984252,❤ [ Inspiration ] Inspired by the Untitled Goo...,❤ [ Enamel pins set ] You will get 3 pieces en...,❤ [ MATERIAL ] : Brooch made of galvanized met...,❤ [ Decoration ] : This pins is perfect for ad...,❤ [ Package ] : This brooches will arrive in a...,...,,,,,,,,,,
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000,Dial size : 12 inches in diameter,"Big , clear representation of the numbers on t...","Sweep and silent movement clock , which requir...",1 year limited warranty on the product from th...,Its Wall Clock design and manufacture by Kangr...,...,,,,,,,,,,
2249696,1165754,"Electro Voice EKX-BRKT15 , Wall Mount Bracket ...",,592,2900.000000,,,,,,...,,,,,,,,,,


In [16]:
data.TITLE = data.TITLE.str.replace('and nbsp ; ', '')
data.DESCRIPTION = data.DESCRIPTION.str.replace('and nbsp ; ', '')

In [17]:
data

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,0,1,2,3,4,...,183,184,185,186,187,188,189,190,191,192
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,,1650,2125.980000,LUXURIOUS and APPEALING : Beautiful custom-mad...,MATERIAL : Luxurious and versatile fabric with...,BLACKOUT CURTAIN : 100 % opaque and heavy prem...,MADE TO PERFECTION : Large eyelets at the top ...,QUALITY ASSURED : Gentle wash with similar col...,...,,,,,,,,,,
1,2673191,Marks and Spencer Girls ' Pyjama Sets T86 2561...,,2755,393.700000,Harry Potter Hedwig Pyjamas ( 6 to 16 years ),100 % cotton ( exclusive of trimmings ),"Ribbing98 % cotton , 2 % elastane",,,...,,,,,,,,,,
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Specifications : Color : Red , Material : Alum...",7537,748.031495,"Loud Dual Tone Trumpet Horn , Compatible With SX4","Electric Air Horn with Compressor , Compatible...","High Quality , High Intensity Horn Sound , Com...","Color : Red , Compatible With SX4","Professional installation required , Compatibl...",...,,,,,,,,,,
3,1594019,ALISHAH Women ' s Cotton Ankle Length Leggings...,AISHAH Women ' s Lycra Cotton Ankel Leggings ....,2996,787.401574,Made By 95 % cotton and 5 % Lycra which gives ...,"Come with Elasticized Waist Band , Slim Fit",Bubble free fabric and High quality steaching ...,Skin-friendly fabric,ANKLE LENGTH LEGGINGS PACK OF 2,...,,,,,,,,,,
4,283658,The United Empire Loyalists : A Chronicle of t...,,6112,598.424000,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249693,2422167,Nike Women ' s As W Ny Df Swsh Hn Kh Bra ( CZ7...,,3009,1181.100000,Material : Polyester,,,,,...,,,,,,,,,,
2249694,2766635,"( 3PCS ) Goose Game Cute Cartoon Enamel Pins ,...",[ Brand ] : XVIEONR [ Product name ] : Fashion...,3413,125.984252,❤ [ Inspiration ] Inspired by the Untitled Goo...,❤ [ Enamel pins set ] You will get 3 pieces en...,❤ [ MATERIAL ] : Brooch made of galvanized met...,❤ [ Decoration ] : This pins is perfect for ad...,❤ [ Package ] : This brooches will arrive in a...,...,,,,,,,,,,
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000,Dial size : 12 inches in diameter,"Big , clear representation of the numbers on t...","Sweep and silent movement clock , which requir...",1 year limited warranty on the product from th...,Its Wall Clock design and manufacture by Kangr...,...,,,,,,,,,,
2249696,1165754,"Electro Voice EKX-BRKT15 , Wall Mount Bracket ...",,592,2900.000000,,,,,,...,,,,,,,,,,


In [18]:
data_bps = data.iloc[:,5:].to_numpy().tolist()

data_bps = [
  [bp for bp in bps if bp != ''] 
  for bps in data_bps 
]

data_bps_len = [len(bps) for bps in data_bps]

temp_bps = pd.Series([bp for bps in data_bps for bp in bps]).astype(str)

temp_bps = temp_bps.str.replace('and nbsp ; ', '')

data_bps_cut_offs = np.cumsum([0]+data_bps_len)

temp_bps = temp_bps.to_list()

replace_bps = [temp_bps[data_bps_cut_offs[i]:data_bps_cut_offs[i+1]] for i in range(len(data_bps_cut_offs)-1)]

replace_bps = pd.DataFrame(replace_bps).fillna('')

data = pd.concat([data.iloc[:,:5], replace_bps], axis=1)

data

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,0,1,2,3,4,...,183,184,185,186,187,188,189,190,191,192
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,,1650,2125.980000,LUXURIOUS and APPEALING : Beautiful custom-mad...,MATERIAL : Luxurious and versatile fabric with...,BLACKOUT CURTAIN : 100 % opaque and heavy prem...,MADE TO PERFECTION : Large eyelets at the top ...,QUALITY ASSURED : Gentle wash with similar col...,...,,,,,,,,,,
1,2673191,Marks and Spencer Girls ' Pyjama Sets T86 2561...,,2755,393.700000,Harry Potter Hedwig Pyjamas ( 6 to 16 years ),100 % cotton ( exclusive of trimmings ),"Ribbing98 % cotton , 2 % elastane",,,...,,,,,,,,,,
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Specifications : Color : Red , Material : Alum...",7537,748.031495,"Loud Dual Tone Trumpet Horn , Compatible With SX4","Electric Air Horn with Compressor , Compatible...","High Quality , High Intensity Horn Sound , Com...","Color : Red , Compatible With SX4","Professional installation required , Compatibl...",...,,,,,,,,,,
3,1594019,ALISHAH Women ' s Cotton Ankle Length Leggings...,AISHAH Women ' s Lycra Cotton Ankel Leggings ....,2996,787.401574,Made By 95 % cotton and 5 % Lycra which gives ...,"Come with Elasticized Waist Band , Slim Fit",Bubble free fabric and High quality steaching ...,Skin-friendly fabric,ANKLE LENGTH LEGGINGS PACK OF 2,...,,,,,,,,,,
4,283658,The United Empire Loyalists : A Chronicle of t...,,6112,598.424000,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249693,2422167,Nike Women ' s As W Ny Df Swsh Hn Kh Bra ( CZ7...,,3009,1181.100000,Material : Polyester,,,,,...,,,,,,,,,,
2249694,2766635,"( 3PCS ) Goose Game Cute Cartoon Enamel Pins ,...",[ Brand ] : XVIEONR [ Product name ] : Fashion...,3413,125.984252,❤ [ Inspiration ] Inspired by the Untitled Goo...,❤ [ Enamel pins set ] You will get 3 pieces en...,❤ [ MATERIAL ] : Brooch made of galvanized met...,❤ [ Decoration ] : This pins is perfect for ad...,❤ [ Package ] : This brooches will arrive in a...,...,,,,,,,,,,
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000,Dial size : 12 inches in diameter,"Big , clear representation of the numbers on t...","Sweep and silent movement clock , which requir...",1 year limited warranty on the product from th...,Its Wall Clock design and manufacture by Kangr...,...,,,,,,,,,,
2249696,1165754,"Electro Voice EKX-BRKT15 , Wall Mount Bracket ...",,592,2900.000000,,,,,,...,,,,,,,,,,


In [19]:
data.to_csv('../Data/2023/train-v3.csv', index=False)
# data.to_csv('../Data/2023/test-v3.csv', index=False)

# data.to_csv('../Data/2021/train-v3.csv', index=False)
# data.to_csv('../Data/2021/test-v3.csv', index=False)